In [ ]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [ ]:
# загружаем изображения для тренировки 
X = []
for filename in os.listdir('/Train'):
    img = load_img('/Train/'+filename)
    img = img.resize( (256, 256), Image.BILINEAR)
    X.append(img_to_array(img))
X = np.array(X, dtype=float)

# нормализуем значения в массиве к диапазону [-1,1] 
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# изменяем каждое следующие изображение от
# предидущего, чтобы нейросеть обучалась быстрее
datagen = ImageDataGenerator(
        shear_range=0.15,
        zoom_range=0.25,
        rotation_range=25,
        horizontal_flip=True)

batch_size = 20
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# начинаем обучение нейросети      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1, steps_per_epoch=20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/200
20/20 [==============================] - 15s 673ms/step - loss: 0.3912
Epoch 2/200
20/20 [==============================] - 13s 673ms/step - loss: 0.0110
Epoch 3/200
20/20 [==============================] - 13s 671ms/step - loss: 0.0073
Epoch 4/200
20/20 [==============================] - 13s 669ms/step - loss: 0.0059
Epoch 5/200
20/20 [==============================] - 13s 666ms/step - loss: 0.0059
Epoch 6/200
20/20 [==============================] - 14s 677ms/step - loss: 0.0056
Epoch 7/200
20/20 [==============================] - 14s 682ms/step - loss: 0.0061
Epoch 8/200
20/20 [==============================] - 14s 678ms/step - loss: 0.0062
Epoch 9/200
20/20 [==============================] - 14s 679ms/step - loss: 0.0065
Epoch 10/200
20/20 [==============================] - 13s 672ms/step - loss: 0.0058
Epoch 11/200
20/20 [==============================] - 13s 673ms/step - loss: 0.0053
Epoch 12/200
20/20 [==============================] - 14s 683ms/step - loss: 0.0073
E

In [ ]:
# сохраняем обученную нейросеть
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [ ]:
# загружаем черно-белые изображения в массив
# и нормализуем значения в массиве к диапазону [-1,1]
colorize = []
for filename in os.listdir('/WBimages/'):
  WBimg = load_img('/WBimages/'+filename)
  WBimg = img.resize( (256, 256), Image.BILINEAR)
  colorize.append(img_to_array(WBimg))

colorize = np.array(colorize, dtype=float)
colorize = rgb2lab(1.0/255*colorize)[:,:,:,0]
colorize = color_me.reshape(colorize.shape+(1,))

# на выходе получили 2 цветовые компоненты и увеличили их на 128
# так как был диапазон [-1,1] а нужен диапазон [-128,127]
output = model.predict(colorize)
output = output * 128

# формируем вспомогательную коллекци, где будет представленно
# цветное изображение и помещаем туда яркостную составляющую
# и цветовые составляющие. Сохраняем цветное изображение
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = colorize[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("/result/img"+str(i)+".png", lab2rgb(cur))